In [1]:
using Plots

In [ ]:
FermiDirac(E) = 1/(exp((E-μ)/kB*T)+1)

In [ ]:
T=300.0
kB=8.617e-5
μ=0.0

In [ ]:
ΔERange=range(-5,5,step=0.01)
ERange = range(-6,6,step=0.1)
output=zeros(length(ΔERange),ERange)

In [ ]:
function integrand(E3,E2)
    return FermiDirac(E3)*(1-FermiDirac(E2))
end

In [ ]:
for i in eachindex(ERange)
    for j in eachindex(ΔERange)
        output[i,l]=integrand(ERange[i],ERange[i]+ΔERange[j])
    end
end

In [ ]:
plot(ERange,ΔERange,output)